# Text Classification

Wir werden hier einen ersten Ansatz für die *Textverarbeitung* in Python kennen lernen, wobei wir die Bilbiothek `scikit-learn` einsetzen werden. Es bedarf in diesem Zusammenhang einer gründlichen Vorverarbeitung der Daten und der *Merkmalsextraktion* ([feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)). Man hat es ferner mit sehr *dünn besetzten Matrizen* (engl. **sparse matrices**) zu tun.

Ziel ist es, Beleidigungen in Diskussionforen zu erkennen. Der zugehörige Datensatz ist im Ordner `data/text_classification` vorhanden.
Er wurde vom [Daten-Repository](https://github.com/ipython-books/cookbook-data) des [Cookbook](https://ipython-books.github.io/)s heruntergeladen.

## Imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection as ms
import sklearn.feature_extraction.text as text
import sklearn.naive_bayes as nb

## Data Preparation

In [2]:
df = pd.read_csv("data/text_classification/troll.csv")

In [3]:
df.head()

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [8]:
df.shape

(3947, 3)

Definition der *Merkmalsmatrix* $\mathbf{X}$ und der *Klassen* (Labels) $\mathbf{y}$:

In [16]:
y = df['Insult']

Die Merkmalsmatrix hingegen ist deutlich schwieriger zu erhalten. `scikit-learn` kann nur mit Zahlen in Matrizen etwas anfangen, so dass man den Text also in eine Matrix umwandeln muss. Diese **Datenvorverarbeitung** (also die *Data Preparation* gemäß [CRISP-DM](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining)) geschieht meist in zwei Schritten:
1. **Tokenizing**: Man extrahiert ein **Vokabular**, d.h. eine Liste von Wörtern, die im Text (in unserem Fall: in den Kommentaren) benutzt wurden
2. **Counting:** Anschließend "zählt" man in jedem Datensatz (auch: *Dokument*), wie oft das jeweilige Wort darin vorkommt. Da es im Allgemeinen sehr viele Wörter gibt und nur die wenigsten davon in einem bestimmten Datensatz (bei uns: in einem Kommentar) wirklich benutzt werden, führt dies zu einer Matrix, die hauptsächlich Nullen enthält (also dünn besetzt ist).

Das gesamte Verfahren wird auch als **Bag of Words** bezeichnet. Mit Hilfe von `scikit-learn` brauchen wir nur zwei Zeilen Code hierfür. Es kommt zum "Zählen" das [Tf-idf](https://de.wikipedia.org/wiki/Tf-idf-Ma%C3%9F) Verfahren zum Einsatz, mit dessen Hilfe zu häufig vorkommende Wörter ("the", "and, etc.) adäquat behandelt werden können. Durch *Tf-idf* wird das Vokabular normalisiert, so dass Wörtern, welche in einer großen Anzahl der Datensätze vorkommen, weniger Gewicht zukommt, als solchen, welche nur in einer geringen Anzahl der Datensätze vorkommen und daher *spezieller* sind.

In [5]:
co = text.CountVectorizer() # Tokenizing + Counting
X = co.fit_transform(df['Comment']) # Transformation aller Kommentare
print(f'In {X.shape[0]} Kommentaren sind {X.shape[1]} verschiedene Wörter')

In 3947 Kommentaren sind 16469 verschiedene Wörter


In [12]:
X.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [13]:
list(X_vec.vocabulary_.items())[:10]

[('you', 16397),
 ('fuck', 5434),
 ('your', 16405),
 ('dad', 3409),
 ('really', 11568),
 ('don', 4075),
 ('understand', 14793),
 ('point', 10754),
 ('xa0', 15720),
 ('it', 7048)]

In [14]:
print("Die Merkmalsmatrix hat ~{0:.2f}% von Null verschiedene Einträge.".format(
          100 * X.nnz / float(X.shape[0] * X.shape[1])))

Die Merkmalsmatrix hat ~0.15% von Null verschiedene Einträge.


Aufteilung in **Trainings-** und **Testdaten**:

In [17]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=.2, random_state = 17)

## Modeling

Als Naive-Bayes-Klassifikator verwenden wir den [Multinomialen Naive Bayes Klassifikator](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_na%C3%AFve_Bayes), welcher die Häufigkeit der Worte als ganzzahligen Wert betrachtet. In der Praxis funktioniert das Modell aber auch mit der Tf-idf-Vektorisierung. Zusätzlich wird eine *Glättung* mit Hilfe eines Parameters $\alpha$ durchgeführt (eine Erklärung des Ansatzes findet man bei der [Stanford University](https://nlp.stanford.edu/courses/cs224n/2001/gruffydd/smoothing.html) und wir kennen ihn als Laplace-Schätzer ...). Weitere Naive-Bayes-Modelle in `scikit-learn` finden sich [hier](https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes).

In [19]:
bnb = ms.GridSearchCV(nb.MultinomialNB(), param_grid={'alpha':np.logspace(-2., 2., 50)})
bnb.fit(X_train, y_train);

## Evaluation

In [20]:
print(f'Die Trefferquote beträgt ca. {round(100*bnb.score(X_test, y_test),2)}%')

Die Trefferquote beträgt ca. 77.85%


In [21]:
insult_class_prob_sorted = bnb.best_estimator_.feature_log_prob_[1, :].argsort()

word_list_insult = np.take(X_vec.get_feature_names_out(), insult_class_prob_sorted[:-30:-1])

print(f'Wörter mit beleidigender Assoziation: \n {word_list_insult}')

Wörter mit beleidigender Assoziation: 
 ['you' 'the' 'to' 'your' 'and' 'are' 'of' 'that' 'it' 'is' 'in' 'like'
 'xa0' 'have' 'on' 'for' 'not' 're' 'just' 'be' 'as' 'this' 'all' 'fuck'
 'get' 'so' 'with' 'what' 'an']


Test mit eigenen Beispielen:

In [23]:
print(bnb.predict(co.transform([
    "You are absolutely right.",
    "This is beyond moronic.",
    "LOL"
    ])))

[0 1 0]
